## Design matrix for sub-trial HMM

In [77]:
prefix = '/home/ines/repositories/'
prefix = '/Users/ineslaranjeira/Documents/Repositories/'

In [78]:
import os
import numpy as np
from one.api import ONE
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import brainbox.behavior.wheel as wh
from scipy.stats import zscore
import concurrent.futures
from brainbox.io.one import SessionLoader
import scipy.interpolate as interpolate
from joblib import Parallel, delayed
from scipy.fftpack import fft, ifft, fftshift
from sklearn.preprocessing import StandardScaler, Normalizer
import gc

# one = ONE(base_url='https://alyx.internationalbrainlab.org')
one = ONE(mode='remote')
# Get my functions
functions_path =  prefix + 'representation_learning_variability/Functions/'
os.chdir(functions_path)
from video_functions import get_lick_on, get_ME, get_lick_times, keypoint_speed_one_camera # get_raw_and_smooth_position, get_dlc_XYs, get_lick_times, get_lick_on, get_pupil_diameter, get_ME, keypoint_speed, pupil_center, nose_tip, left_paw, right_paw
# from design_functions import lick_rate, wheel_velocity, wheel_displacement, pupil_diam, cont_bin
# from data_processing import fast_wavelet_morlet_convolution_parallel, resample_common_time

In [3]:
import ibllib
print(ibllib.__file__)

/Users/ineslaranjeira/Documents/Repositories/ibllib/ibllib/__init__.py


# Design Matrix V6 14 Jul 2025

- Same as V5 but with paws and other minor changes

### Get sessions which pass QC

In [ ]:
# Load preprocessed data
results_path = prefix + 'representation_learning_variability/paper-individuality/'
filename = str(results_path + '1_bwm_qc_07-10-2025')
bwm_query = pickle.load(open(filename, "rb"))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/paper-individuality/final_lp_qc07-10-2025'

In [3]:
# Load preprocessed data
results_path = prefix + 'representation_learning_variability/Video and wheel/Video QC/'
# filename = str(results_path + '1_bwm_qc_07-10-2025')
filename = str(results_path + 'final_lp_qc07-10-2025')

bwm_query = pickle.load(open(filename, "rb"))

### Parameters

In [4]:
bin_size = 0.1  # seconds
bin_size = 0.017  # np.round(1/60, 3)  # No binning, number indicates sampling rate
video_type = 'left'    
first_90 = False
lp = True

# Wavelet decomposition
f = np.array([.25, .5, 1, 2, 4, 8, 16])
omega0 = 5

sessions =  bwm_query['eid'].unique()

In [ ]:
def get_feature_event_times(dlc, dlc_t, features):
    """
    Detect events from the dlc traces. Based on the standard deviation between frames
    :param dlc: dlc pqt table
    :param dlc_t: dlc times
    :param features: features to consider
    :return:
    """

    for i, feat in enumerate(features):
        f = dlc[feat]
        threshold = np.nanstd(np.diff(f)) / 4
        if i == 0:
            events = np.where(np.abs(np.diff(f)) > threshold)[0]
        else:
            events = np.r_[events, np.where(np.abs(np.diff(f)) > threshold)[0]]

    return dlc_t[np.unique(events)]


def merge_licks(poses, features, common_fs):
    
    # Define total duration (max of both videos)
    duration_sec = max(list(poses['leftCamera']['times'])[-1], list(poses['rightCamera']['times'])[-1])  # in seconds

    # Set common sampling rate (high rather than low)
    t_common = np.arange(0, duration_sec, 1/common_fs)  # uniform timestamps
    
    lick_trace_left = np.zeros_like(t_common, dtype=int)
    lick_trace_right = np.zeros_like(t_common, dtype=int)

    left_lick_times = get_feature_event_times(poses['leftCamera'], poses['leftCamera']['times'], features)
    right_lick_times = get_feature_event_times(poses['rightCamera'], poses['rightCamera']['times'], features)

    # Round licks to nearest timestamp in t_common
    left_indices = np.searchsorted(t_common, left_lick_times)
    right_indices = np.searchsorted(t_common, right_lick_times)

    # Set licks to 1
    lick_trace_left[left_indices[left_indices < len(t_common)]] = 1
    lick_trace_right[right_indices[right_indices < len(t_common)]] = 1

    combined_licks = np.maximum(lick_trace_left, lick_trace_right)
    
    return t_common, combined_licks 


def interpolate_nans(pose):
    
    # threshold (in seconds) above which we will not interpolate nans, but keep them
    # (for long stretches interpolation may not be appropriate)
    nan_thresh = 1
    fr = 60
    interp_pose = pose.interpolate()

    # don't interpolate long strings of nans
    t = np.diff(1 * np.isnan(interp_pose))
    begs = np.where(t == 1)[0]
    ends = np.where(t == -1)[0]
    if begs.shape[0] > ends.shape[0]:
        begs = begs[:ends.shape[0]]
    for b, e in zip(begs, ends):
        if (e - b) > (fr * nan_thresh):
            interp_pose[(b + 1):(e + 1)] = np.nan  # offset by 1 due to earlier diff
        
    return interp_pose


def get_speed(poses, times, camera, split, feature='paw_r'):
    """
    FIXME Document and add unit test!

    :param dlc: dlc pqt table
    :param dlc_t: dlc time points
    :param camera: camera type e.g 'left', 'right', 'body'
    :param feature: dlc feature to compute speed over
    :return:
    """
    SAMPLING = {'left': 60,
                'right': 150,
                'body': 30}
    RESOLUTION = {'left': 2,
                  'right': 1,
                  'body': 1}

    speeds = {}
    interpolated_x = interpolate_nans(poses[f'{feature}_x'])
    interpolated_y = interpolate_nans(poses[f'{feature}_y'])
    x = interpolated_x / RESOLUTION[camera]
    y = interpolated_y / RESOLUTION[camera]

    # get speed in px/sec [half res]
    s = ((np.diff(x) ** 2 + np.diff(y) ** 2) ** .5) * SAMPLING[camera]

    dt = np.diff(times)
    tv = times[:-1] + dt / 2


    # Calculate velocity for x and y separately if split is true
    if split == True:
        s_x = np.diff(x) * SAMPLING[camera]
        s_y = np.diff(y) * SAMPLING[camera]
        speeds[video_type] = [times, s_x, s_y]

        # interpolate over original time scale
        if tv.size > 1:
            ifcn_x = interpolate.interp1d(tv, s_x, fill_value="extrapolate")
            ifcn_y = interpolate.interp1d(tv, s_y, fill_value="extrapolate")

            speeds[video_type] = [times, ifcn_x(times), ifcn_y(times)]

    else:
        # Speed vector is given by the Pitagorean theorem
        s = ((np.diff(x)**2 + np.diff(y)**2)**.5) * SAMPLING[camera]
        speeds[video_type] = [times, s]

        # interpolate over original time scale
        if tv.size > 1:
            ifcn = interpolate.interp1d(tv, s, fill_value="extrapolate")
            
            speeds[video_type] = [times, ifcn(times)]
    return speeds        


# def get_lick_times(poses, features, combine=False, video_type='left'):
    
#     if combine:    
#         # combine licking events from left and right cam
#         lick_times = []
#         for video_type in ['right','left']:
#             camera_name = str(video_type+'Camera')
#             camera_licks = get_feature_event_times(poses[camera_name], 
#                                                  poses[camera_name]['times'], features)        
#             lick_times.append(camera_licks)
        
#         lick_times = np.array(sorted(np.concatenate(lick_times)))
        
#     else:
#         lick_times = get_feature_event_times(poses[video_type], 
#                                              poses[video_type]['times'], features)     
        
#     return lick_times


def resample_common_time(reference_time, timestamps, data, kind, fill_gaps=None):
    
    # # t = np.arange(t_init, t_end, 1 / freq)  # Evenly resample at frequency
    # if reference_time[-1] > timestamps[-1]:
    #     reference_time = reference_time[:-1]  # Occasionally due to precision errors the last sample may be outside of range.
    yinterp = interpolate.interp1d(timestamps, data, kind=kind, fill_value='extrapolate')(reference_time)
    
    if fill_gaps:
        #  Find large gaps and forward fill @fixme This is inefficient
        gaps, = np.where(np.diff(timestamps) >= fill_gaps)

        for i in gaps:
            yinterp[(reference_time >= timestamps[i]) & (reference_time < timestamps[i + 1])] = data[i]
            
    return yinterp, reference_time


# def resample_common_time(reference_time, timestamps, data, kind, fill_gaps=None):
#     # Function inspired on wh.interpolate from here: https://github.com/int-brain-lab/ibllib/blob/master/brainbox/behavior/wheel.py#L28 
#     # t = np.arange(t_init, t_end, 1 / freq)  # Evenly resample at frequency
#     if reference_time[-1] > timestamps[-1]:
#         reference_time = reference_time[:-1]  # Occasionally due to precision errors the last sample may be outside of range.
#     yinterp = interpolate.interp1d(timestamps, data, kind=kind, fill_value='extrapolate')(reference_time)
    
#     if fill_gaps:
#         #  Find large gaps and forward fill @fixme This is inefficient
#         gaps, = np.where(np.diff(timestamps) >= fill_gaps)

#         for i in gaps:
#             yinterp[(reference_time >= timestamps[i]) & (reference_time < timestamps[i + 1])] = data[i]
            
#     return yinterp, reference_time


def morlet_conj_ft(omega_vals, omega0):
    """
    Computes the conjugate Fourier transform of the Morlet wavelet.
    
    Parameters:
    - w: Angular frequency values (array or scalar)
    - omega0: Dimensionless Morlet wavelet parameter
    
    Returns:
    - out: Conjugate Fourier transform of the Morlet wavelet
    """
    
    return np.pi**(-1/4) * np.exp(-0.5 * (omega_vals - omega0)**2)


def fast_wavelet_morlet_convolution_parallel(x, f, omega0, dt):
    """
    Fast Morlet wavelet transform using parallel computation.

    Args:
        x (array): 1D array of projection values to transform.
        f (array): Center frequencies of the wavelet frequency channels (Hz).
        omega0 (float): Dimensionless Morlet wavelet parameter.
        dt (float): Sampling time (seconds).

    Returns:
        amp (array): Wavelet amplitudes.
        W (array): Wavelet coefficients (complex-valued, optional).
    """
    N = len(x)
    L = len(f)
    amp = np.zeros((L, N))
    Q = np.zeros((L, N))

    # Ensure N is even
    if N % 2 == 1:
        x = np.append(x, 0)
        N += 1
        test = True
    else:
        test = False

    # Add zero padding to x
    # Zero padding serves to compensate for the fact that the kernel does not have the same size as 
    # 
    x = np.concatenate((np.zeros(N // 2), x, np.zeros(N // 2)))
    M = N
    N = len(x)

    # Compute scales
    scales = (omega0 + np.sqrt(2 + omega0**2)) / (4 * np.pi * f)
    # angular frequencies to compute FT for (depends on sampling frequency); is as long as N 
    omega_vals = 2 * np.pi * np.arange(-N // 2, N // 2) / (N * dt)  

    # Fourier transform of x; shift folds it around zero so that it is more interpretable (frequencies at the right of nyquist become negative)
    x_hat = fftshift(fft(x))

    # Index for truncation to recover the actual x without padding
    if test:
        idx = np.arange(M // 2, M // 2 + M - 1)
    else:
        idx = np.arange(M // 2, M // 2 + M)

    # Function for parallel processing
    def process_frequency(i):
        # Take the Morlet conjugate of the Fourier transform
        m = morlet_conj_ft(-omega_vals * scales[i], omega0)
        # Convolution on the Fourier domain (as opposed to time domain in DWT)
        conv = m * x_hat
        # Inverse Fourier transform (normalized?)
        # q are the wavelet coefficients; normalized to ensure the energy of the wavelet is preserved across different scales
        q = ifft(conv) * np.sqrt(scales[i])
        # Recover q without padding
        q = q[idx]
        amp_row = np.abs(q) * np.pi**-0.25 * np.exp(0.25 * (omega0 - np.sqrt(omega0**2 + 2))**2) / np.sqrt(2 * scales[i])
        return amp_row, q

    # Parallel processing
    results = Parallel(n_jobs=-1)(delayed(process_frequency)(i) for i in range(L))

    for i, (amp_row, q) in enumerate(results):
        amp[i, :] = amp_row
        Q[i, :] = q

    return amp, Q, x_hat


## Check sessions missing

In [ ]:
# Loop through animals
function_name = process_design_matrix
sessions =  bwm_query['eid'].unique()
data_path =prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
os.chdir(data_path)
files = os.listdir()
sessions_to_process = []

for s, sess in enumerate(sessions):
    file_path = one.eid2path(sess)
    mouse_name = file_path.parts[8]
    filename = "design_matrix_" + str(sess) + '_'  + mouse_name
    if filename not in files:
        sessions_to_process.append((sess))

len(sessions_to_process)
        

208

## Without session loader

In [16]:
def process_design_matrix(session):
    
    file_path = one.eid2path(session)
    mouse_name = file_path.parts[8]

    """ LOAD VARIABLES """
    sl = SessionLoader(eid=session, one=one)
    sl.load_trials()
    sl.load_motion_energy()
    sl.load_wheel()
    # sl.load_session_data()
    # poses = sl.pose  #TODO: sl.load_pose(views=views, tracker='lightningPose')
    
    # Check if all data is available
    #if np.sum(sl.data_info['is_loaded']) >=4:
        
    # Motion energy
    me = sl.motion_energy
    me_time = np.array(me['leftCamera']['times'])
    motion_energy = np.array(zscore(me['leftCamera']['whiskerMotionEnergy'], nan_policy='omit'))
    
    # Licks  # TODO: licks should change when LP is available on session loader
    # Define total duration (max of both videos)
    duration_sec = me_time[-1]  # in seconds
    # Set common sampling rate (high rather than low)
    common_fs = 150
    t_common = np.arange(0, duration_sec, 1/common_fs)  # uniform timestamps
    lick_trace_left = np.zeros_like(t_common, dtype=int)
    lick_trace_right = np.zeros_like(t_common, dtype=int)
    lp = True
    left_lick_times = get_lick_times(one, session, lp, combine=False, video_type='left')
    right_lick_times = get_lick_times(one, session, lp, combine=False, video_type='right')
    # Round licks to nearest timestamp in t_common
    left_indices = np.searchsorted(t_common, left_lick_times)
    right_indices = np.searchsorted(t_common, right_lick_times)
    # Set licks to 1
    lick_trace_left[left_indices[left_indices < len(t_common)]] = 1
    lick_trace_right[right_indices[right_indices < len(t_common)]] = 1
    licks = np.maximum(lick_trace_left, lick_trace_right)

    # Paws
    ephys=True
    keypoint='paw_r'  # paw = 'r' if cam == 'left' else 'l'
    split=True
    speeds = keypoint_speed_one_camera(one, session, ephys, video_type, keypoint, split, lp)
    paw_time = speeds['left'][0]
    paw_x = speeds['left'][1]
    paw_y = speeds['left'][2]
    
    # Wheel
    wheel = sl.wheel
    wheel_time = np.array(wheel['times'])
    wheel_vel = np.array(wheel['velocity'])
    
    """ COMMON TIMESTAMPS AND RESAMPLING"""
    # Use reference time, truncate and resample
    onset = np.max([np.min(me_time), np.min(t_common), np.min(wheel_time), np.min(paw_time)])
    offset = np.min([np.max(me_time), np.max(t_common), np.max(wheel_time), np.max(paw_time)])
    
    common_fs = 60
    # Set common sampling rate (high rather than low)
    reference_time = np.arange(onset, offset, 1/common_fs)  # uniform timestamps

    me_time = me_time[np.where((me_time >= onset) & (me_time < offset))[0]]
    motion_energy = motion_energy[np.where((me_time >= onset) & (me_time < offset))[0]]
    donwsampled_me, corrected_me_t = resample_common_time(reference_time, me_time, motion_energy, kind='linear', fill_gaps=None)

    wheel_time = wheel_time[np.where((wheel_time >= onset) & (wheel_time < offset))]
    wheel_vel = wheel_vel[np.where((wheel_time >= onset) & (wheel_time < offset))]
    donwsampled_wheel, corrected_wheel_t = resample_common_time(reference_time, wheel_time, wheel_vel, kind='linear', fill_gaps=None)
    
    licks_time = t_common[np.where((t_common >= onset) & (t_common < offset))]
    licks = licks[np.where((licks_time >= onset) & (licks_time < offset))]
    donwsampled_lick, corrected_lick_t = resample_common_time(reference_time, licks_time, licks, kind='nearest', fill_gaps=None)

    paw_time = paw_time[np.where((paw_time >= onset) & (paw_time < offset))]
    paw_x = paw_x[np.where((paw_time >= onset) & (paw_time < offset))]
    donwsampled_paw_x, corrected_paw_x_t = resample_common_time(reference_time, paw_time, paw_x, kind='linear', fill_gaps=None)
    paw_y = paw_y[np.where((paw_time >= onset) & (paw_time < offset))]
    donwsampled_paw_y, corrected_paw_y_t = resample_common_time(reference_time, paw_time, paw_y, kind='linear', fill_gaps=None)

    # Check integrity of data
    assert (corrected_me_t == corrected_wheel_t).all()
    assert (corrected_wheel_t == corrected_lick_t).all()
    assert (corrected_lick_t == corrected_paw_x_t).all()
    assert (corrected_paw_x_t == corrected_paw_y_t).all()

    # Wavelet decomposition of wheel velocity
    dt = np.round(np.mean(np.diff(corrected_wheel_t)), 3)
    amp, Q, x_hat = fast_wavelet_morlet_convolution_parallel(donwsampled_wheel, f, omega0, dt)

    """ GROUP DATA INTO DESIGN MATRIX """
    design_matrix = pd.DataFrame(columns=['Bin', 'avg_wheel_vel', 'whisker_me', 'Lick count', 'paw_x', 'paw_y'])
    design_matrix['Bin'] = corrected_me_t.copy()
    design_matrix['Lick count'] = donwsampled_lick.copy()
    design_matrix['avg_wheel_vel'] = donwsampled_wheel.copy()
    design_matrix['whisker_me'] = donwsampled_me.copy()
    design_matrix['paw_x'] = donwsampled_paw_x.copy()
    design_matrix['paw_y'] = donwsampled_paw_y.copy()

    # Wavelet transforms
    for i, frequency in enumerate(f):
        # Create new column with frequency
        design_matrix[str(frequency)] = design_matrix['Bin'] * np.nan
        design_matrix[str(frequency)] = amp[i, :]
        
    """ LOAD TRIAL DATA """
    session_trials = sl.trials
    session_start = list(session_trials['goCueTrigger_times'])[0]

    # Get time of last unbiased trial
    unbiased = session_trials.loc[session_trials['probabilityLeft']==0.5]
    time_trial_90 = list(unbiased['stimOff_times'])[-1]

    if first_90 == True:
        # Keep only first 90 trials
        design_matrix = design_matrix.loc[(design_matrix['Bin'] < time_trial_90) & 
                                            (design_matrix['Bin'] > session_start)]
        use_trials = session_trials.loc[session_trials['stimOff_times'] < time_trial_90]
    else:
        design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]
        use_trials = session_trials.copy()

    """ STANDARDIZE DATA """
    training_set = np.array(design_matrix).copy()[:, 1:]
    # Standardization using StandardScaler
    scaler = StandardScaler()
    std_design_matrix = scaler.fit_transform(training_set)
    # Keep licks unnormalized
    std_design_matrix[:, 2] = training_set[:, 2]  

    """ SAVE DATA """       
    # Save unnormalized design matrix
    data_path =  prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
    filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
    design_matrix.to_parquet(filename, compression='gzip')  

    # Save standardized design matrix
    data_path =  prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
    filename = data_path + "standardized_design_matrix_" + str(session) + '_'  + mouse_name
    np.save(filename, std_design_matrix)

    # Save trials
    data_path =  prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
    filename = data_path + "session_trials_" + str(session) + '_'  + mouse_name
    use_trials.to_parquet(filename, compression='gzip')  
    
    del design_matrix, std_design_matrix, use_trials, sl
    gc.collect()

def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)

## With session loader

In [64]:
def process_design_matrix(session):    

    file_path = one.eid2path(session)
    if prefix == '/home/ines/repositories/':
        mouse_name = file_path.parts[8]
    else:
        mouse_name = file_path.parts[7]

    """ LOAD VARIABLES """
    sl = SessionLoader(eid=session, one=one)
    sl.load_pose(views=['left', 'right'], tracker='lightningPose')
    sl.load_session_data(trials=True, wheel=True, motion_energy=True)

    # Check if all data is available
    if np.sum(sl.data_info['is_loaded']) >=4:
        
        # Motion energy
        me = sl.motion_energy
        me_time = np.array(me['leftCamera']['times'])
        motion_energy = np.array(zscore(me['leftCamera']['whiskerMotionEnergy'], nan_policy='omit'))
        
        # Licks 
        features = ['tongue_end_l_x', 'tongue_end_l_y','tongue_end_r_x', 'tongue_end_r_y']        
        common_fs = 150 
        poses = sl.pose
        licks_time, licks = merge_licks(poses, features, common_fs)

        # Paws
        paw_speeds = get_speed(poses['leftCamera'], poses['leftCamera']['times'], 'left', True, feature='paw_r')
        paw_time = np.array(paw_speeds['left'][0])
        paw_x = paw_speeds['left'][1]
        paw_y = paw_speeds['left'][2]
        
        # Wheel
        wheel = sl.wheel
        wheel_time = np.array(wheel['times'])
        wheel_vel = np.array(wheel['velocity'])
        
        """ COMMON TIMESTAMPS AND RESAMPLING"""
        # Use reference time, truncate and resample
        onset = np.max([np.min(poses['leftCamera']['times']), np.min(poses['rightCamera']['times']), np.min(wheel['times'])])
        offset = np.min([np.max(poses['leftCamera']['times']), np.max(poses['rightCamera']['times']), np.max(wheel['times'])])
        
        common_fs = 60
        # Set common sampling rate (high rather than low)
        reference_time = np.arange(onset, offset, 1/common_fs)  # uniform timestamps
        
        motion_energy = motion_energy[np.where((me_time >= onset) & (me_time <= offset))[0]]
        me_time = me_time[np.where((me_time >= onset) & (me_time <= offset))[0]]
        donwsampled_me, corrected_me_t = resample_common_time(reference_time, me_time, motion_energy, kind='linear', fill_gaps=None)

        wheel_vel = wheel_vel[np.where((wheel_time >= onset) & (wheel_time < offset))]
        wheel_time = wheel_time[np.where((wheel_time >= onset) & (wheel_time < offset))]
        donwsampled_wheel, corrected_wheel_t = resample_common_time(reference_time, wheel_time, wheel_vel, kind='linear', fill_gaps=None)

        licks = licks[np.where((licks_time >= onset) & (licks_time < offset))]
        licks_time = licks_time[np.where((licks_time >= onset) & (licks_time < offset))]
        donwsampled_lick, corrected_lick_t = resample_common_time(reference_time, licks_time, licks, kind='nearest', fill_gaps=None)

        paw_time = paw_time[np.where((paw_time >= onset) & (paw_time < offset))]
        paw_x = paw_x[np.where((paw_time >= onset) & (paw_time < offset))]
        donwsampled_paw_x, corrected_paw_x_t = resample_common_time(reference_time, paw_time, paw_x, kind='linear', fill_gaps=None)
        paw_y = paw_y[np.where((paw_time >= onset) & (paw_time < offset))]
        donwsampled_paw_y, corrected_paw_y_t = resample_common_time(reference_time, paw_time, paw_y, kind='linear', fill_gaps=None)

        # Check integrity of data
        assert (corrected_me_t == corrected_wheel_t).all()
        assert (corrected_wheel_t == corrected_lick_t).all()
        assert (corrected_lick_t == corrected_paw_x_t).all()
        assert (corrected_paw_x_t == corrected_paw_y_t).all()

        # Wavelet decomposition of wheel velocity
        dt = np.round(np.mean(np.diff(corrected_wheel_t)), 3)
        amp, Q, x_hat = fast_wavelet_morlet_convolution_parallel(donwsampled_wheel, f, omega0, dt)

        """ GROUP DATA INTO DESIGN MATRIX """
        design_matrix = pd.DataFrame(columns=['Bin', 'avg_wheel_vel', 'whisker_me', 'Lick count', 'paw_x', 'paw_y'])
        design_matrix['Bin'] = corrected_me_t.copy()
        design_matrix['Lick count'] = donwsampled_lick.copy()
        design_matrix['avg_wheel_vel'] = donwsampled_wheel.copy()
        design_matrix['whisker_me'] = donwsampled_me.copy()
        design_matrix['paw_x'] = donwsampled_paw_x.copy()
        design_matrix['paw_y'] = donwsampled_paw_y.copy()

        # Wavelet transforms
        for i, frequency in enumerate(f):
            # Create new column with frequency
            design_matrix[str(frequency)] = design_matrix['Bin'] * np.nan
            design_matrix[str(frequency)] = amp[i, :]
            
        # """ LOAD TRIAL DATA """
        session_trials = sl.trials
        session_start = list(session_trials['goCueTrigger_times'])[0]

        # Get time of last unbiased trial
        unbiased = session_trials.loc[session_trials['probabilityLeft']==0.5]
        time_trial_90 = list(unbiased['stimOff_times'])[-1]

        if first_90 == True:
            # Keep only first 90 trials
            design_matrix = design_matrix.loc[(design_matrix['Bin'] < time_trial_90) & 
                                                (design_matrix['Bin'] > session_start)]
            use_trials = session_trials.loc[session_trials['stimOff_times'] < time_trial_90]
        else:
            design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]
            use_trials = session_trials.copy()

        """ STANDARDIZE DATA """
        training_set = np.array(design_matrix).copy()[:, 1:]
        # Standardization using StandardScaler
        scaler = StandardScaler()
        std_design_matrix = scaler.fit_transform(training_set)
        # Keep licks unnormalized
        std_design_matrix[:, 2] = training_set[:, 2]  

        """ SAVE DATA """       
        # Save unnormalized design matrix
        data_path =  prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
        filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
        design_matrix.to_parquet(filename, compression='gzip')  

        # Save standardized design matrix
        data_path =  prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
        filename = data_path + "standardized_design_matrix_" + str(session) + '_'  + mouse_name
        np.save(filename, std_design_matrix)

        # Save trials
        data_path =  prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
        filename = data_path + "session_trials_" + str(session) + '_'  + mouse_name
        use_trials.to_parquet(filename, compression='gzip')  
        
        del design_matrix, std_design_matrix, use_trials, sl
        gc.collect()

    else:
        print('Data missing for session '+session)  


def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)
    

## Check sessions missing

In [66]:
# Loop through animals
function_name = process_design_matrix
sessions =  bwm_query['eid'].unique()
data_path =prefix + 'representation_learning_variability/DATA/Sub-trial/Design matrix/v6_21Jul2025/' + str(bin_size) + '/'
os.chdir(data_path)
files = os.listdir()
sessions_to_process = []

for s, sess in enumerate(sessions):
    file_path = one.eid2path(sess)
    mouse_name = file_path.parts[8]
    filename = "design_matrix_" + str(sess) + '_'  + mouse_name
    if filename not in files:
        sessions_to_process.append((sess))

len(sessions_to_process)
        

218

## Process sessions in series

In [67]:
for s, session in enumerate(sessions_to_process[:10]):
    process_design_matrix(session)

(S3) /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-21/2020-08-19/001/alf/_ibl_rightCamera.lightningPose.pqt: 100%|██████████| 318M/318M [00:51<00:00, 6.21MB/s] 


2025-07-23 11:37:40 INFO     one.py:1288 Loading trials data


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2025-07-23 11:37:41 INFO     one.py:1288 Loading wheel data


(S3) /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-21/2020-08-19/001/alf/_ibl_wheel.timestamps.npy: 100%|██████████| 14.4M/14.4M [00:08<00:00, 1.66MB/s]
(S3) /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-21/2020-08-19/001/alf/_ibl_wheel.position.npy: 100%|██████████| 14.4M/14.4M [00:06<00:00, 2.32MB/s]


...

2025-07-23 11:42:54 INFO     one.py:1288 Loading pupil data



/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "2025-06-18", ""
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
(S3) /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-39/2021-05-10/001/alf/#2025-06-18#/_ibl_leftCamera.features.pqt: 100%|██████████| 4.07M/4.07M [00:01<00:00, 2.88MB/s]

2025-07-23 11:42:57 WARNING  one.py:1292 Could not load pupil data.



/var/folders/nt/d2j3zp9d1xzb8wgfrw81j0c40000gn/T/ipykernel_18163/464933178.py:238: ComplexWarning: Casting complex values to real discards the imaginary part
  Q[i, :] = q
local file size mismatch on dataset: angelakilab/Subjects/NYU-39/2021-05-11/001/alf/_ibl_leftCamera.lightningPose.pqt
(S3) /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-39/2021-05-11/001/alf/_ibl_leftCamera.lightningPose.pqt: 100%|██████████| 114M/114M [00:19<00:00, 5.85MB/s] 
(S3) /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-39/2021-05-11/001/alf/_ibl_rightCamera.lightningPose.pqt:  56%|█████▌    | 142M/254M [00:23<00:18, 6.16MB/s] 


KeyboardInterrupt: 

In [57]:
session = sessions[33]  

file_path = one.eid2path(session)
if prefix == '/home/ines/repositories/':
    mouse_name = file_path.parts[8]
else:
    mouse_name = file_path.parts[7]

""" LOAD VARIABLES """
sl = SessionLoader(eid=session, one=one)
sl.load_pose(views=['left', 'right'], tracker='lightningPose')
sl.load_session_data(trials=True, wheel=True, motion_energy=True)

# Check if all data is available
if np.sum(sl.data_info['is_loaded']) >=4:
    
    # Motion energy
    me = sl.motion_energy
    me_time = np.array(me['leftCamera']['times'])
    motion_energy = np.array(zscore(me['leftCamera']['whiskerMotionEnergy'], nan_policy='omit'))
    
    # Licks 
    features = ['tongue_end_l_x', 'tongue_end_l_y','tongue_end_r_x', 'tongue_end_r_y']        
    common_fs = 150 
    poses = sl.pose
    licks_time, licks = merge_licks(poses, features, common_fs)

    # Paws
    paw_speeds = get_speed(poses['leftCamera'], poses['leftCamera']['times'], 'left', True, feature='paw_r')
    paw_time = np.array(paw_speeds['left'][0])
    paw_x = paw_speeds['left'][1]
    paw_y = paw_speeds['left'][2]
    
    # Wheel
    wheel = sl.wheel
    wheel_time = np.array(wheel['times'])
    wheel_vel = np.array(wheel['velocity'])
    
    """ COMMON TIMESTAMPS AND RESAMPLING"""
    # Use reference time, truncate and resample
    onset = np.max([np.min(poses['leftCamera']['times']), np.min(poses['rightCamera']['times']), np.min(wheel['times'])])
    offset = np.min([np.max(poses['leftCamera']['times']), np.max(poses['rightCamera']['times']), np.max(wheel['times'])])
    
    common_fs = 60
    # Set common sampling rate (high rather than low)
    reference_time = np.arange(onset, offset, 1/common_fs)  # uniform timestamps
    
    motion_energy = motion_energy[np.where((me_time >= onset) & (me_time <= offset))[0]]
    me_time = me_time[np.where((me_time >= onset) & (me_time <= offset))[0]]
    donwsampled_me, corrected_me_t = resample_common_time(reference_time, me_time, motion_energy, kind='linear', fill_gaps=None)

    wheel_vel = wheel_vel[np.where((wheel_time >= onset) & (wheel_time < offset))]
    wheel_time = wheel_time[np.where((wheel_time >= onset) & (wheel_time < offset))]
    donwsampled_wheel, corrected_wheel_t = resample_common_time(reference_time, wheel_time, wheel_vel, kind='linear', fill_gaps=None)

    licks = licks[np.where((licks_time >= onset) & (licks_time < offset))]
    licks_time = licks_time[np.where((licks_time >= onset) & (licks_time < offset))]
    donwsampled_lick, corrected_lick_t = resample_common_time(reference_time, licks_time, licks, kind='nearest', fill_gaps=None)

    paw_time = paw_time[np.where((paw_time >= onset) & (paw_time < offset))]
    paw_x = paw_x[np.where((paw_time >= onset) & (paw_time < offset))]
    donwsampled_paw_x, corrected_paw_x_t = resample_common_time(reference_time, paw_time, paw_x, kind='linear', fill_gaps=None)
    paw_y = paw_y[np.where((paw_time >= onset) & (paw_time < offset))]
    donwsampled_paw_y, corrected_paw_y_t = resample_common_time(reference_time, paw_time, paw_y, kind='linear', fill_gaps=None)

    # Check integrity of data
    assert (corrected_me_t == corrected_wheel_t).all()
    assert (corrected_wheel_t == corrected_lick_t).all()
    assert (corrected_lick_t == corrected_paw_x_t).all()
    assert (corrected_paw_x_t == corrected_paw_y_t).all()

    # Wavelet decomposition of wheel velocity
    dt = np.round(np.mean(np.diff(corrected_wheel_t)), 3)
    amp, Q, x_hat = fast_wavelet_morlet_convolution_parallel(donwsampled_wheel, f, omega0, dt)

    """ GROUP DATA INTO DESIGN MATRIX """
    design_matrix = pd.DataFrame(columns=['Bin', 'avg_wheel_vel', 'whisker_me', 'Lick count', 'paw_x', 'paw_y'])
    design_matrix['Bin'] = corrected_me_t.copy()
    design_matrix['Lick count'] = donwsampled_lick.copy()
    design_matrix['avg_wheel_vel'] = donwsampled_wheel.copy()
    design_matrix['whisker_me'] = donwsampled_me.copy()
    design_matrix['paw_x'] = donwsampled_paw_x.copy()
    design_matrix['paw_y'] = donwsampled_paw_y.copy()

    # Wavelet transforms
    for i, frequency in enumerate(f):
        # Create new column with frequency
        design_matrix[str(frequency)] = design_matrix['Bin'] * np.nan
        design_matrix[str(frequency)] = amp[i, :]
        
    # """ LOAD TRIAL DATA """
    session_trials = sl.trials
    session_start = list(session_trials['goCueTrigger_times'])[0]
    # Get time of last unbiased trial
    unbiased = session_trials.loc[session_trials['probabilityLeft']==0.5]
    time_trial_90 = list(unbiased['stimOff_times'])[-1]
    design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]
    use_trials = session_trials.copy()

    """ STANDARDIZE DATA """
    training_set = np.array(design_matrix).copy()[:, 1:]
    # Standardization using StandardScaler
    scaler = StandardScaler()
    std_design_matrix = scaler.fit_transform(training_set)
    # Keep licks unnormalized
    std_design_matrix[:, 2] = training_set[:, 2]  


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2023-04-20"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2023-04-20"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2025-07-23 18:14:54 INFO     one.py:1288 Loading trials data


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-03-03"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2025-07-23 18:14:55 INFO     one.py:1288 Loading wheel data
2025-07-23 18:14:56 INFO     one.py:1288 Loading motion_energy data


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "2023-04-20", "2025-05-31"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "2023-04-20", "2025-06-01"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2025-07-23 18:14:57 INFO     one.py:1288 Loading pupil data


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "2025-06-04", "2023-04-20"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)


2025-07-23 18:14:57 WARNING  one.py:1292 Could not load pupil data.


/var/folders/nt/d2j3zp9d1xzb8wgfrw81j0c40000gn/T/ipykernel_25023/3920472031.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  Q[i, :] = q


# Design Matrix V5 15 Jan 2025

- Includes wavelet decomposition in addition to wheel velocity, licks and whisker motion energy
- Lower bin size

### Get sessions which pass QC

In [2]:
# Load preprocessed data
results_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
filename = str(results_path + 'custom_qc01-07-2025')
custom_qc = pickle.load(open(filename, "rb"))

### Parameters

In [3]:
bin_size = 0.1  # seconds
bin_size = 0.017  # np.round(1/60, 3)  # No binning, number indicates sampling rate
video_type = 'left'    
first_90 = False

# Wavelet decomposition
f = np.array([.25, .5, 1, 2, 4, 8, 16])
omega0 = 5

sessions =  custom_qc['eid'].unique()

In [4]:
def resample_common_time(reference_time, timestamps, data, kind, fill_gaps=None):
    # Function inspired on wh.interpolate from here: https://github.com/int-brain-lab/ibllib/blob/master/brainbox/behavior/wheel.py#L28 
    # t = np.arange(t_init, t_end, 1 / freq)  # Evenly resample at frequency
    if reference_time[-1] > timestamps[-1]:
        reference_time = reference_time[:-1]  # Occasionally due to precision errors the last sample may be outside of range.
    yinterp = interpolate.interp1d(timestamps, data, kind=kind, fill_value='extrapolate')(reference_time)
    
    if fill_gaps:
        #  Find large gaps and forward fill @fixme This is inefficient
        gaps, = np.where(np.diff(timestamps) >= fill_gaps)

        for i in gaps:
            yinterp[(reference_time >= timestamps[i]) & (reference_time < timestamps[i + 1])] = data[i]
            
    return yinterp, reference_time


def morlet_conj_ft(omega_vals, omega0):
    """
    Computes the conjugate Fourier transform of the Morlet wavelet.
    
    Parameters:
    - w: Angular frequency values (array or scalar)
    - omega0: Dimensionless Morlet wavelet parameter
    
    Returns:
    - out: Conjugate Fourier transform of the Morlet wavelet
    """
    
    return np.pi**(-1/4) * np.exp(-0.5 * (omega_vals - omega0)**2)


def fast_wavelet_morlet_convolution_parallel(x, f, omega0, dt):
    """
    Fast Morlet wavelet transform using parallel computation.

    Args:
        x (array): 1D array of projection values to transform.
        f (array): Center frequencies of the wavelet frequency channels (Hz).
        omega0 (float): Dimensionless Morlet wavelet parameter.
        dt (float): Sampling time (seconds).

    Returns:
        amp (array): Wavelet amplitudes.
        W (array): Wavelet coefficients (complex-valued, optional).
    """
    N = len(x)
    L = len(f)
    amp = np.zeros((L, N))
    Q = np.zeros((L, N))

    # Ensure N is even
    if N % 2 == 1:
        x = np.append(x, 0)
        N += 1
        test = True
    else:
        test = False

    # Add zero padding to x
    # Zero padding serves to compensate for the fact that the kernel does not have the same size as 
    # 
    x = np.concatenate((np.zeros(N // 2), x, np.zeros(N // 2)))
    M = N
    N = len(x)

    # Compute scales
    scales = (omega0 + np.sqrt(2 + omega0**2)) / (4 * np.pi * f)
    # angular frequencies to compute FT for (depends on sampling frequency); is as long as N 
    omega_vals = 2 * np.pi * np.arange(-N // 2, N // 2) / (N * dt)  

    # Fourier transform of x; shift folds it around zero so that it is more interpretable (frequencies at the right of nyquist become negative)
    x_hat = fftshift(fft(x))

    # Index for truncation to recover the actual x without padding
    if test:
        idx = np.arange(M // 2, M // 2 + M - 1)
    else:
        idx = np.arange(M // 2, M // 2 + M)

    # Function for parallel processing
    def process_frequency(i):
        # Take the Morlet conjugate of the Fourier transform
        m = morlet_conj_ft(-omega_vals * scales[i], omega0)
        # Convolution on the Fourier domain (as opposed to time domain in DWT)
        conv = m * x_hat
        # Inverse Fourier transform (normalized?)
        # q are the wavelet coefficients; normalized to ensure the energy of the wavelet is preserved across different scales
        q = ifft(conv) * np.sqrt(scales[i])
        # Recover q without padding
        q = q[idx]
        amp_row = np.abs(q) * np.pi**-0.25 * np.exp(0.25 * (omega0 - np.sqrt(omega0**2 + 2))**2) / np.sqrt(2 * scales[i])
        return amp_row, q

    # Parallel processing
    results = Parallel(n_jobs=-1)(delayed(process_frequency)(i) for i in range(L))

    for i, (amp_row, q) in enumerate(results):
        amp[i, :] = amp_row
        Q[i, :] = q

    return amp, Q, x_hat

In [5]:
def process_design_matrix(session):

    file_path = one.eid2path(session)
    mouse_name = file_path.parts[8]

    """ LOAD VARIABLES """
    # Licks
    video_time, lick_on = get_lick_on(one, session, video_type)

    # Motion energy of whisker pad
    me_times, me = get_ME(one, session, video_type)    
    motion_energy = zscore(me, nan_policy='omit') 
    assert len(video_time) == len(me_times), print('ME and lick times do not match!')

    # Wheel
    wheel = one.load_object(session, 'wheel', collection='alf')
    pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)

    """ COMMON TIMESTAMPS AND RESAMPLING"""
    # Use video time as reference, truncate to fit wheel_times
    reference_time = video_time.copy()
    onset = np.max([np.min(video_time), np.min(wheel_times)])
    offset = np.min([np.max(video_time), np.max(wheel_times)])
    reference_time = reference_time[np.where(video_time > onset)[0][0]:np.where(video_time < offset)[0][-1]]

    # Truncate all other variables based on onset and offset, before downsampling
    lick_on = lick_on[np.where(video_time > onset)[0][0]:np.where(video_time < offset)[0][-1]]
    motion_energy = motion_energy[np.where(me_times > onset)[0][0]:np.where(me_times < offset)[0][-1]]
    pos = pos[np.where(wheel_times > onset)[0][0]:np.where(wheel_times < offset)[0][-1]]
    wheel_times = wheel_times[np.where(wheel_times > onset)[0][0]:np.where(wheel_times < offset)[0][-1]]

    # Downsample
    donwsampled_wheel, corrected_wheel_t = resample_common_time(reference_time, wheel_times, pos, kind='linear', fill_gaps=None)

    # Check data integrity
    assert len(reference_time) == len(corrected_wheel_t)
    assert len(reference_time) == len(donwsampled_wheel)
    assert len (reference_time) == len(motion_energy)
    assert len (reference_time) == len(lick_on)
        
    # Wavelet decomposition of wheel velocity
    wheel_vel = np.diff(donwsampled_wheel)
    dt = np.round(np.mean(np.diff(corrected_wheel_t)), 3)
    amp, _, _ = fast_wavelet_morlet_convolution_parallel(wheel_vel, f, omega0, dt)

    """ GROUP DATA INTO DESIGN MATRIX """
    design_matrix = pd.DataFrame(columns=['Bin', 'avg_wheel_vel', 'whisker_me', 'Lick count'])
    design_matrix['Bin'] = reference_time.copy()
    design_matrix['Lick count'] = lick_on.copy()
    design_matrix['avg_wheel_vel'][:-1] = wheel_vel.copy()
    design_matrix['whisker_me'] = motion_energy.copy()

    # Wavelet transforms
    for i, frequency in enumerate(f):
        # Create new column with frequency
        design_matrix[str(frequency)] = design_matrix['Bin'] * np.nan
        design_matrix[str(frequency)][:-1] = amp[i, :]
        
    """ LOAD TRIAL DATA """
    session_trials = one.load_object(session, obj='trials', namespace='ibl')
    session_trials = session_trials.to_df()
    session_start = list(session_trials['goCueTrigger_times'])[0]

    # Get time of last unbiased trial
    unbiased = session_trials.loc[session_trials['probabilityLeft']==0.5]
    time_trial_90 = list(unbiased['stimOff_times'])[-1]

    if first_90 == True:
        # Keep only first 90 trials
        design_matrix = design_matrix.loc[(design_matrix['Bin'] < time_trial_90) & 
                                            (design_matrix['Bin'] > session_start)]
        use_trials = session_trials.loc[session_trials['stimOff_times'] < time_trial_90]
    else:
        design_matrix = design_matrix.loc[(design_matrix['Bin'] > session_start)]
        use_trials = session_trials.copy()

    """ STANDARDIZE DATA """
    training_set = np.array(design_matrix).copy()[:, 1:]
    # Standardization using StandardScaler
    scaler = StandardScaler()
    std_design_matrix = scaler.fit_transform(training_set)
    # Keep licks unnormalized
    std_design_matrix[:, 2] = training_set[:, 2]  

    """ SAVE DATA """       
    # Save unnormalized design matrix
    data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v5_15Jan2025/' + str(bin_size) + '/'
    filename = data_path + "design_matrix_" + str(session) + '_'  + mouse_name
    design_matrix.to_parquet(filename, compression='gzip')  

    # Save standardized design matrix
    data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v5_15Jan2025/' + str(bin_size) + '/'
    filename = data_path + "standardized_design_matrix_" + str(session) + '_'  + mouse_name
    np.save(filename, std_design_matrix)

    # Save trials
    data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v5_15Jan2025/' + str(bin_size) + '/'
    filename = data_path + "session_trials_" + str(session) + '_'  + mouse_name
    use_trials.to_parquet(filename, compression='gzip')  
    
    del design_matrix, std_design_matrix, use_trials
    gc.collect()
    
def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)


## Check sessions missing

In [12]:
# Loop through animals
function_name = process_design_matrix
sessions =  custom_qc['eid'].unique()
data_path = '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v5_15Jan2025/' + str(bin_size) + '/'
os.chdir(data_path)
files = os.listdir()
sessions_to_process = []

for s, sess in enumerate(sessions):
    file_path = one.eid2path(sess)
    mouse_name = file_path.parts[8]
    filename = "design_matrix_" + str(sess) + '_'  + mouse_name
    if filename not in files:
        sessions_to_process.append((sess))
        

## Process sessions in series

In [11]:
for s, session in enumerate(sessions_to_process):
    process_design_matrix(session)

/tmp/ipykernel_29322/3471468597.py:100: ComplexWarning: Casting complex values to real discards the imaginary part
  Q[i, :] = q
/tmp/ipykernel_29322/3753953485.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  design_matrix['avg_wheel_vel'][:-1] = wheel_vel.copy()
/tmp/ipykernel_29322/3753953485.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  design_matrix[str(frequency)][:-1] = amp[i, :]
/tmp/ipykernel_29322/3753953485.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

## Process sessions missing in parallel

In [18]:
# Loop through animals
function_name = process_design_matrix
bundle_size = 2
loop_times = len(sessions_to_process)//bundle_size
for l in range(loop_times):
    start_idx = l*bundle_size
    end_idx = l*bundle_size+bundle_size
    use_sessions = sessions_to_process[start_idx:end_idx]
    parallel_process_data(use_sessions, function_name)  # TODO: figure how to use params inside these functions


KeyboardInterrupt: 

# Design Matrix V4 9 Dec 2024

- Same as V4 but only includes wheel velocity, licks and whisker motion energy

### Parameters

In [2]:
bin_size = 0.1  # seconds
video_type = 'left'    

In [3]:
# data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Brainwidemap/'
# # data_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/DATA/Brainwidemap/'

# os.chdir(data_path)https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-022-29600-1/MediaObjects/41467_2022_29600_MOESM1_ESM.pdf
ext_qc_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick01-03-2025", "rb"))
# Filter based on custom lick qc and some key video qcs
final_custom_qc = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                (lick_custom['videoLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                (lick_custom['ratio']>5) &
                (lick_custom['_videoLeft_pin_state'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &
                (lick_custom['_videoLeft_framerate'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &  
                (lick_custom['_videoLeft_camera_times'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS')) &
                (lick_custom['_videoLeft_dropped_frames'].apply(lambda x: (isinstance(x, list) and True in x) or x == 'PASS' or x == None)) &
                (lick_custom['_videoLeft_timestamps'].isin([True, 'PASS', np.nan]))                   
                ]
    
custom_qc_eids = final_custom_qc['eid'].unique() 


In [4]:
def process_design_matrix(session):
    bin_size = 0.1
    video_type = 'left'

    file_path = one.eid2path(session)
    mouse_name = file_path.parts[8]

    session_trials = one.load_object(session, obj='trials', namespace='ibl')
    session_trials = session_trials.to_df()

    # Licks
    session_licks = get_lick_times(one, session, combine=False, video_type=video_type)

    # Wheel
    wheel = one.load_object(session, 'wheel', collection='alf')
    pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)
    
    # Motion energy of whisker pad
    me_times, motion_energy = get_ME(one, session, video_type)       
    motion_energy = zscore(motion_energy, nan_policy='omit') 


    # Initialize dataframe
    licking = lick_rate(bin_size, session_licks, session_trials)
    wheel_vel = wheel_velocity(bin_size, wheel_times, pos, session_trials)
    # wheel_disp = wheel_displacement(wheel_times, pos, session_trials, bin_size, onset_subtraction=True)
    whisker_me = cont_bin(me_times, motion_energy, session_trials, bin_size)

    # Rename column names
    whisker_me = whisker_me.rename(columns={'Values':'whisker_me'})
    
    # Merge all tables
    all_metrics = wheel_vel[['Bin', 'avg_wheel_vel']].dropna().merge(licking[['Bin', 'Lick count']], on='Bin', how='outer')
    # all_metrics = all_metrics.merge(wheel_disp[['Bin', 'wheel_final']], on='Bin', how='outer')
    all_metrics = all_metrics.merge(whisker_me[['Bin', 'whisker_me']], on='Bin', how='outer')

    # Remove Nans before adding more data or else this breaks (not sure if this is really tru, but worked like this)
    all_metrics['Lick count'] = all_metrics['Lick count'].replace(np.NaN, 0)
    all_metrics = all_metrics.dropna().drop_duplicates()

    # Save design matrix
    data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v4_9Dec2024/' + str(bin_size) + '/'
    os.chdir(data_path)
    pickle.dump(all_metrics, open("design_matrix_" + str(session) + '_'  + mouse_name + '_' + str(bin_size), "wb"))


def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)

In [10]:
# Loop through sessions one by one
function_name = process_design_matrix
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v4_9Dec2024/' + str(bin_size) + '/'
os.chdir(data_path)
files = os.listdir()

for s, sess in enumerate(custom_qc_eids):
    file_path = one.eid2path(sess)
    mouse_name = file_path.parts[8]

    filename = "design_matrix_" + str(sess) + '_'  + mouse_name + '_' + str(bin_size)
    if filename not in files:
        process_design_matrix(sess)
        # parallel_process_data([sess], function_name)

# Design Matrix V4 5 Jul 2024

- V3 includes data to be used on 'movement detector' 2ARHMMs
- Includes no pupil data because it is not trustworthy
- Includes the two paws

### Get sessions from mice of interest

In [6]:
# Choose a session with good QC
data_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/'
# data_path = '/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Video and wheel/'

os.chdir(data_path)
pass_qc = pickle.load(open(data_path + "good_brainwide_sessions_metadata", "rb"))
pass_qc = pickle.load(open(data_path + "good_dlc_bwm_sessions_metadata", "rb"))
# mouse_names = pass_qc['subject_nickname'].unique()

### Parameters

In [2]:
bin_size = 0.1  # seconds
video_type = 'left'    

## Compute frame rate per session - not necessary if using the custom/extended QC

In [20]:
frame_rate = pass_qc.copy()
frame_rate['frame_rate'] = np.zeros(len(pass_qc)) * np.nan
for s, session in enumerate(pass_qc['session_uuid']):
    try:
        
        mouse_name = pass_qc['subject_nickname'][s]
        dlc_times, XYs = get_dlc_XYs(one, session, view=video_type, likelihood_thresh=0.9)
        frt = 1/np.mean(np.diff(dlc_times))
        
        # Save
        frame_rate.loc[(frame_rate['subject_nickname']==mouse_name) & 
                       (frame_rate['session_uuid']==session), 'frame_rate'] = frt

    except:
       print(session)

# Save design matrix
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/frame_rate/'
os.chdir(data_path)
pickle.dump(frame_rate, open("frame_rate", "wb"))

# Run code in parallel

In [17]:
# Loop through animals
function_name = process_design_matrix
sessions = pass_qc['session_uuid']
# sessions = list(pass_qc.loc[pass_qc['subject_nickname']=='CSHL045', 'session_uuid'])
data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v4_5Jul2024/' + str(bin_size) + '/'
os.chdir(data_path)
files = os.listdir()
for s, sess in enumerate(sessions):
    mouse_name = list(pass_qc.loc[pass_qc['session_uuid']==sess, 'subject_nickname'])[0]
    filename = "design_matrix_" + str(sess) + '_'  + mouse_name + '_' + str(bin_size)
    if filename not in files:
        parallel_process_data([sess], function_name)

d32876dd-8303-4720-8e7e-20678dc2fd71
72982282-e493-45ee-87ce-aa45cb3a3ec1
6434f2f5-6bce-42b8-8563-d93d493613a2
64e3fb86-928c-4079-865c-b364205b502e
15948667-747b-4702-9d53-354ac70e9119
288bfbf3-3700-4abe-b6e4-130b5c541e61
fc43390d-457e-463a-9fd4-b94a0a8b48f5
3d59aa1a-b4ba-47fe-b9cf-741b5fdb0c7b
6364ff7f-6471-415a-ab9e-632a12052690


local md5 mismatch on dataset: cortexlab/Subjects/KS014/2019-12-07/001/alf/_ibl_trials.goCueTrigger_times.npy


Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/cortexlab/Subjects/KS014/2019-12-07/001/alf/_ibl_trials.goCueTrigger_times.bc2892ec-a74f-4697-ab1f-42ccf0a00443.npy Bytes: 5384


100%|██████████| 0.00513458251953125/0.00513458251953125 [00:00<00:00,  1.70s/it]
local md5 mismatch on dataset: cortexlab/Subjects/KS023/2019-12-06/001/alf/_ibl_trials.goCueTrigger_times.npy


Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/cortexlab/Subjects/KS023/2019-12-06/001/alf/_ibl_trials.goCueTrigger_times.624b4901-d15f-40b4-97af-f9f72d74a51c.npy Bytes: 5352


...

100%|██████████| 0.00441741943359375/0.00441741943359375 [00:00<00:00,  1.09s/it]


Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/wittenlab/Subjects/ibl_witten_29/2021-06-09/002/alf/#2024-07-15#/_ibl_trials.stimOff_times.ec4d2e83-6966-4e22-b32f-8d68c44ebe2f.npy Bytes: 4632


100%|██████████| 0.00441741943359375/0.00441741943359375 [00:00<00:00,  2.16it/s]
100%|██████████| 0.04532623291015625/0.04532623291015625 [00:00<00:00,  6.49s/it]


Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/wittenlab/Subjects/ibl_witten_29/2021-06-09/002/alf/#2024-07-15#/_ibl_trials.stimOffTrigger_times.910276d1-dd24-43cd-ac4b-c5efc58b5c58.npy Bytes: 4632


100%|██████████| 0.00441741943359375/0.00441741943359375 [00:00<00:00,  1.31it/s]


In [9]:
# Loop through animals
function_name = process_design_matrix
sessions = pass_qc['session_uuid']
loop_times = int(np.ceil(len(sessions)/5))  # tried to run 10 sessions in parallel but it breaks

for l in range(loop_times):
    start_idx = l*5
    end_idx = l*5+5
    use_sessions = sessions[start_idx:end_idx]
    parallel_process_data(use_sessions, function_name)

64e3fb86-928c-4079-865c-b364205b502e
d32876dd-8303-4720-8e7e-20678dc2fd71
6434f2f5-6bce-42b8-8563-d93d493613a2
15948667-747b-4702-9d53-354ac70e9119
72982282-e493-45ee-87ce-aa45cb3a3ec1
288bfbf3-3700-4abe-b6e4-130b5c541e61
fc43390d-457e-463a-9fd4-b94a0a8b48f5
3d59aa1a-b4ba-47fe-b9cf-741b5fdb0c7b


local md5 mismatch on dataset: churchlandlab/Subjects/CSHL049/2020-01-11/001/alf/_ibl_trials.stimOff_times.npy
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/churchlandlab/Subjects/CSHL049/2020-01-11/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 4.60k/4.60k [00:00<00:00, 7.21kB/s]


6364ff7f-6471-415a-ab9e-632a12052690


local md5 mismatch on dataset: cortexlab/Subjects/KS014/2019-12-03/001/alf/_ibl_trials.goCueTrigger_times.npy
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/cortexlab/Subjects/KS014/2019-12-03/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 4.38k/4.38k [00:00<00:00, 7.10kB/s]
local md5 mismatch on dataset: cortexlab/Subjects/KS023/2019-12-08/001/alf/_ibl_trials.goCueTrigger_times.npy
local md5 mismatch on dataset: cortexlab/Subjects/KS023/2019-12-06/001/alf/_ibl_trials.goCueTrigger_times.npy
local md5 mismatch on dataset: cortexlab/Subjects/KS014/2019-12-07/001/alf/_ibl_trials.goCueTrigger_times.npy
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/cortexlab/Subjects/KS023/2019-12-06/001/alf/_ibl_trials.goCueTrigger_times.npy:   0%|          | 0.00/5.35k [00:00<?, ?B/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/cortexlab/Subjects/KS023/2019-12-06/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 5.35k/5.35k [00:00<00:00, 9.97kB/s]

/

: 

: 

In [5]:
def process_design_matrix(session):
    bin_size = 0.1
    try:
        
        mouse_name = list(pass_qc.loc[pass_qc['session_uuid']==session, 'subject_nickname'])[0]

        session_trials = one.load_object(session, obj='trials', namespace='ibl')
        session_trials = session_trials.to_df()

        # Licks
        session_licks = get_lick_times(one, session, combine=False, video_type=video_type)

        # Wheel
        wheel = one.load_object(session, 'wheel', collection='alf')
        pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)

        # # Pupil diameter
        # pupil_dia_raw, pupil_dia_smooth, _, _ = (
        #     get_raw_and_smooth_position(one, session, video_type, ephys=True, position_function=get_pupil_diameter))
        dlc_times, XYs = get_dlc_XYs(one, session, view=video_type, likelihood_thresh=0.9)
        
        # # Pupil position
        # pupil_X, pupil_Y = pupil_center(XYs)
        
        # Motion energy of whisker pad
        me_times, motion_energy = get_ME(one, session, video_type)       
        motion_energy = zscore(motion_energy, nan_policy='omit') 
        
        # Nose positions
        nose_X, nose_Y = nose_tip(XYs)

        # Left paw positions
        left_paw_X, left_paw_Y = left_paw(XYs)
        
        # Right paw positions
        right_paw_X, right_paw_Y = right_paw(XYs)
        
        # Get session values
        # session_length = list(session_trials['stimOff_times'][-1:])[0]  # TODO: this might not work if stimOff times are missing
        # n_bins = int(np.floor(session_length/bin_size))
        # onsets = session_trials['stimOn_times']

        # Initialize dataframe
        licking = lick_rate(bin_size, session_licks, session_trials)
        wheel_vel = wheel_velocity(bin_size, wheel_times, pos, session_trials)
        wheel_disp = wheel_displacement(wheel_times, pos, session_trials, bin_size, onset_subtraction=True)
        # pupil_dia = pupil_diam(dlc_times, pupil_dia_smooth, session_trials, bin_size, onset_subtraction=True)
        whisker_me = cont_bin(me_times, motion_energy, session_trials, bin_size)
        left_X = cont_bin(dlc_times, left_paw_X, session_trials, bin_size)
        left_Y = cont_bin(dlc_times, left_paw_Y, session_trials, bin_size)
        right_X = cont_bin(dlc_times, right_paw_X, session_trials, bin_size)
        right_Y = cont_bin(dlc_times, right_paw_Y, session_trials, bin_size)
        # pupil_X_pos = cont_bin(dlc_times, pupil_X, session_trials, bin_size)
        # pupil_Y_pos = cont_bin(dlc_times, pupil_Y, session_trials, bin_size)
        nose_X_pos = cont_bin(dlc_times, nose_X, session_trials, bin_size)
        nose_Y_pos = cont_bin(dlc_times, nose_Y, session_trials, bin_size)
        
        # Rename column names
        # pupil_dia = pupil_dia.rename(columns={'pupil_final':'pupil_diameter'})
        whisker_me = whisker_me.rename(columns={'Values':'whisker_me'})
        left_X = left_X.rename(columns={'Values':'left_X'})
        left_Y = left_Y.rename(columns={'Values':'left_Y'})
        right_X = right_X.rename(columns={'Values':'right_X'})
        right_Y = right_Y.rename(columns={'Values':'right_Y'})
        # pupil_X_pos = pupil_X_pos.rename(columns={'Values':'pupil_X'})
        # pupil_Y_pos = pupil_Y_pos.rename(columns={'Values':'pupil_Y'})   
        nose_X_pos = nose_X_pos.rename(columns={'Values':'nose_X'})
        nose_Y_pos = nose_Y_pos.rename(columns={'Values':'nose_Y'})     
        
        # Merge all tables
        all_metrics = wheel_vel[['Bin', 'avg_wheel_vel']].dropna().merge(licking[['Bin', 'Lick count']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(wheel_disp[['Bin', 'wheel_final']], on='Bin', how='outer')
        # all_metrics = all_metrics.merge(pupil_dia[['Bin', 'pupil_diameter']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(whisker_me[['Bin', 'whisker_me']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_X[['Bin', 'left_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_Y[['Bin', 'left_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(right_X[['Bin', 'right_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(right_Y[['Bin', 'right_Y']], on='Bin', how='outer')
        # all_metrics = all_metrics.merge(pupil_X_pos[['Bin', 'pupil_X']], on='Bin', how='outer')
        # all_metrics = all_metrics.merge(pupil_Y_pos[['Bin', 'pupil_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_X_pos[['Bin', 'nose_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_Y_pos[['Bin', 'nose_Y']], on='Bin', how='outer')

        # Remove Nans before adding more data or else this breaks (not sure if this is really tru, but worked like this)
        all_metrics['Lick count'] = all_metrics['Lick count'].replace(np.NaN, 0)
        all_metrics = all_metrics.dropna().drop_duplicates()

        # Save design matrix
        data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v4_5Jul2024/' + str(bin_size) + '/'
        os.chdir(data_path)
        pickle.dump(all_metrics, open("design_matrix_" + str(session) + '_'  + mouse_name + '_' + str(bin_size), "wb"))

    except:
       print(session)

def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)

# Design Matrix V3

- V3 includes data to be used on 'movement detector' 2ARHMMs

In [4]:
def process_design_matrix(session):
    bin_size = 0.1
    try:
        
        mouse_name = list(pass_qc.loc[pass_qc['session_uuid']==session, 'subject_nickname'])[0]

        session_trials = one.load_object(session, obj='trials', namespace='ibl')
        session_trials = session_trials.to_df()

        # Licks
        session_licks = get_lick_times(one, session, combine=False, video_type=video_type)

        # Wheel
        wheel = one.load_object(session, 'wheel', collection='alf')
        pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)

        # Pupil diameter
        pupil_dia_raw, pupil_dia_smooth, _, _ = (
            get_raw_and_smooth_position(one, session, video_type, ephys=True, position_function=get_pupil_diameter))
        dlc_times, XYs = get_dlc_XYs(one, session, view=video_type, likelihood_thresh=0.9)
        
        # Pupil position
        pupil_X, pupil_Y = pupil_center(XYs)
        
        # Motion energy of whisker pad
        me_times, motion_energy = get_ME(one, session, video_type)       
        motion_energy = zscore(motion_energy, nan_policy='omit') 
        
        # Nose positions
        nose_X, nose_Y = nose_tip(XYs)

        # Left paw positions
        left_paw_X, left_paw_Y = left_paw(XYs)
        
        # Right paw positions
        right_paw_X, right_paw_Y = right_paw(XYs)
        
        # Get session values
        session_length = list(session_trials['stimOff_times'][-1:])[0]  # TODO: this might not work if stimOff times are missing
        n_bins = int(np.floor(session_length/bin_size))
        onsets = session_trials['stimOn_times']

        # Initialize dataframe
        licking = lick_rate(bin_size, session_licks, session_trials)
        wheel_vel = wheel_velocity(bin_size, wheel_times, pos, session_trials)
        wheel_disp = wheel_displacement(wheel_times, pos, session_trials, bin_size, onset_subtraction=True)
        pupil_dia = pupil_diam(dlc_times, pupil_dia_smooth, session_trials, bin_size, onset_subtraction=True)
        whisker_me = cont_bin(me_times, motion_energy, session_trials, bin_size)
        left_X = cont_bin(dlc_times, left_paw_X, session_trials, bin_size)
        left_Y = cont_bin(dlc_times, left_paw_Y, session_trials, bin_size)
        right_X = cont_bin(dlc_times, right_paw_X, session_trials, bin_size)
        right_Y = cont_bin(dlc_times, right_paw_Y, session_trials, bin_size)
        pupil_X_pos = cont_bin(dlc_times, pupil_X, session_trials, bin_size)
        pupil_Y_pos = cont_bin(dlc_times, pupil_Y, session_trials, bin_size)
        nose_X_pos = cont_bin(dlc_times, nose_X, session_trials, bin_size)
        nose_Y_pos = cont_bin(dlc_times, nose_Y, session_trials, bin_size)
        
        # Rename column names
        pupil_dia = pupil_dia.rename(columns={'pupil_final':'pupil_diameter'})
        whisker_me = whisker_me.rename(columns={'Values':'whisker_me'})
        left_X = left_X.rename(columns={'Values':'left_X'})
        left_Y = left_Y.rename(columns={'Values':'left_Y'})
        right_X = right_X.rename(columns={'Values':'right_X'})
        right_Y = right_Y.rename(columns={'Values':'right_Y'})
        pupil_X_pos = pupil_X_pos.rename(columns={'Values':'pupil_X'})
        pupil_Y_pos = pupil_Y_pos.rename(columns={'Values':'pupil_Y'})   
        nose_X_pos = nose_X_pos.rename(columns={'Values':'nose_X'})
        nose_Y_pos = nose_Y_pos.rename(columns={'Values':'nose_Y'})     
        
        # Merge all tables
        all_metrics = wheel_vel[['Bin', 'avg_wheel_vel']].dropna().merge(licking[['Bin', 'Lick count']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(wheel_disp[['Bin', 'wheel_final']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_dia[['Bin', 'pupil_diameter']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(whisker_me[['Bin', 'whisker_me']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_X[['Bin', 'left_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_Y[['Bin', 'left_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(right_X[['Bin', 'right_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(right_Y[['Bin', 'right_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_X_pos[['Bin', 'pupil_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_Y_pos[['Bin', 'pupil_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_X_pos[['Bin', 'nose_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_Y_pos[['Bin', 'nose_Y']], on='Bin', how='outer')

        # Remove Nans before adding more data or else this breaks (not sure if this is really tru, but worked like this)
        all_metrics['Lick count'] = all_metrics['Lick count'].replace(np.NaN, 0)
        all_metrics = all_metrics.dropna().drop_duplicates()

        # Save design matrix
        data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v3_5Jul2024/' + str(bin_size) + '/'
        os.chdir(data_path)
        pickle.dump(all_metrics, open("design_matrix_" + str(session) + '_'  + mouse_name + '_' + str(bin_size), "wb"))

    except:
       print(session)

def parallel_process_data(sessions, function_name):
    with concurrent.futures.ThreadPoolExecutor() as executor:

        # Process each chunk in parallel
        executor.map(function_name, sessions)

# Design matrix V1

In [4]:
for s, session in enumerate(pass_qc['session_uuid'][0:1]):
    # TODO: as it is this takes ~10 minutes!! Mostly because of smoothing, outlier removal, ... of timeseries
    
    try:
        mouse_name = pass_qc['subject_nickname'][s]
        
        # Trials data
        session_trials = one.load_object(session, obj='trials', namespace='ibl')
        session_trials = session_trials.to_df()

        # Licks
        session_licks = get_lick_times(one, session, combine=False, video_type=video_type)

        # Wheel
        wheel = one.load_object(session, 'wheel', collection='alf')
        pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)

        # Pupil diameter
        pupil_dia_raw, pupil_dia_smooth, _, _ = (
            get_raw_and_smooth_position(one, session, video_type, ephys=True, position_function=get_pupil_diameter))
        pupil_t, XYs = get_dlc_XYs(one, session, view=video_type, likelihood_thresh=0.9)

        # Motion energy of whisker pad
        me_times, motion_energy = get_ME(one, session, video_type)       
        #motion_energy = zscore(motion_energy, nan_policy='omit') 

        # Nose velocity
        nose_speeds = keypoint_speed(one, session, True, 'nose_tip', True)
        nose_times = nose_speeds['left'][0][1:]
        nose_speed_X = nose_speeds['left'][1]
        nose_speed_Y = nose_speeds['left'][2]

        # Left paw velocity
        left_p_speeds = keypoint_speed(one, session, True, 'paw_r', True)
        left_p_times = left_p_speeds['left'][0][1:]
        left_p_speed_X = left_p_speeds['left'][1]
        left_p_speed_Y = left_p_speeds['left'][2]

        # Pupil velocity
        pupil_speeds = keypoint_speed(one, session, True, pupil_center, True)
        pupil_times = pupil_speeds['left'][0][1:]
        pupil_speed_X = pupil_speeds['left'][1]
        pupil_speed_Y = pupil_speeds['left'][2]
        
        
        session_length = list(session_trials['stimOff_times'][-1:])[0]  # TODO: this might not work if stimOff times are missing
        n_bins = int(np.floor(session_length/bin_size))
        onsets = session_trials['stimOn_times']

        # Initialize dataframe
        licking = lick_rate(bin_size, session_licks, session_trials)
        wheel_vel = wheel_velocity(bin_size, wheel_times, pos, session_trials)
        wheel_disp = wheel_displacement(wheel_times, pos, session_trials, bin_size, onset_subtraction=False)
        pupil = pupil_diam(pupil_t, pupil_dia_smooth, session_trials, bin_size, onset_subtraction=True)
        whisker_me = cont_bin(me_times, motion_energy, session_trials, bin_size)
        nose_vel_X = cont_bin(nose_times, nose_speed_X, session_trials, bin_size)
        nose_vel_Y = cont_bin(nose_times, nose_speed_Y, session_trials, bin_size)
        left_vel_X = cont_bin(left_p_times, left_p_speed_X, session_trials, bin_size)
        left_vel_Y = cont_bin(left_p_times, left_p_speed_Y, session_trials, bin_size)
        pupil_vel_X = cont_bin(pupil_times, pupil_speed_X, session_trials, bin_size)
        pupil_vel_Y = cont_bin(pupil_times, pupil_speed_Y, session_trials, bin_size)
        
        pupil = pupil.rename(columns={'pupil_final':'pupil_diameter'})
        whisker_me = whisker_me.rename(columns={'Values':'whisker_me'})
        nose_vel_X = nose_vel_X.rename(columns={'Values':'nose_speed_X'})
        nose_vel_Y = nose_vel_Y.rename(columns={'Values':'nose_speed_Y'})
        left_vel_X = left_vel_X.rename(columns={'Values':'l_paw_speed_X'})
        left_vel_Y = left_vel_Y.rename(columns={'Values':'l_paw_speed_Y'})
        pupil_vel_X = pupil_vel_X.rename(columns={'Values':'pupil_speed_X'})
        pupil_vel_Y = pupil_vel_Y.rename(columns={'Values':'pupil_speed_Y'})
        
        all_metrics = wheel_vel[['Bin', 'avg_wheel_vel']].dropna().merge(licking[['Bin', 'Lick count']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(wheel_disp[['Bin', 'Onset times']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil[['Bin', 'pupil_diameter']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(whisker_me[['Bin', 'whisker_me']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_vel_X[['Bin', 'nose_speed_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_vel_Y[['Bin', 'nose_speed_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_vel_X[['Bin', 'l_paw_speed_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_vel_Y[['Bin', 'l_paw_speed_Y']], on='Bin', how='outer')

        # Remove Nans before adding more data or else this breaks (not sure if this is really tru, but worked like this)
        all_metrics['Lick count'] = all_metrics['Lick count'].replace(np.NaN, 0)
        all_metrics = all_metrics.dropna().drop_duplicates()

        new = all_metrics.merge(pupil_vel_X[['Bin', 'pupil_speed_X']], on='Bin', how='outer')
        new = new.merge(pupil_vel_Y[['Bin', 'pupil_speed_Y']], on='Bin', how='outer')

        all_metrics = new.copy()
        all_metrics = all_metrics.dropna().drop_duplicates()

        # Transform licks
        def anscombe_transform(x):
            return 2 * np.sqrt(x + 3/8)
        all_metrics['Gaussian_licks'] = anscombe_transform(all_metrics['Lick count'])
        all_metrics['Gaussian_licks'] = np.sqrt(all_metrics['Lick count'])

        # Save design matrix
        data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix' + str(bin_size) + '/'
        os.chdir(data_path)
        pickle.dump(all_metrics, open("design_matrix_" + str(session) + '_' + mouse_name + str(bin_size), "wb"))
        
    except:
        print(session)

/home/ines/miniconda3/envs/iblenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,


7af49c00-63dd-4fed-b2e0-1b3bd945b20b


In [13]:
for s, session in enumerate(pass_qc['session_uuid']):
    # TODO: as it is this takes ~10 minutes!! Mostly because of smoothing, outlier removal, ... of timeseries
    
    try:
        mouse_name = pass_qc['subject_nickname'][s]

        session_trials = one.load_object(session, obj='trials', namespace='ibl')
        session_trials = session_trials.to_df()

        # Get session values
        session_length = list(session_trials['stimOff_times'][-1:])[0]  # TODO: this might not work if stimOff times are missing
        n_bins = int(np.floor(session_length/bin_size))
        onsets = session_trials['stimOn_times']

        # Licks
        session_licks = get_lick_times(session, combine=False, video_type=video_type)

        # Wheel
        wheel = one.load_object(session, 'wheel', collection='alf')
        pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)

        # Pupil diameter
        pupil_dia_raw, pupil_dia_smooth, _, _ = (
            get_raw_and_smooth_position(session, video_type, ephys=True, position_function=get_pupil_diameter))
        dlc_times, XYs = get_dlc_XYs(session, view=video_type, likelihood_thresh=0.9)
        
        # Pupil position
        pupil_X, pupil_Y = pupil_center(XYs)
        
        # Motion energy of whisker pad
        me_times, motion_energy = get_ME(session, video_type)       
        #motion_energy = zscore(motion_energy, nan_policy='omit') 

        # Nose velocity
        nose_speeds = keypoint_speed(session, True, 'nose_tip', False)
        nose_times = nose_speeds['left'][0][1:]
        nose_speed = nose_speeds['left'][1]
        #nose_X, nose_Y = nose_tip(XYs)
        
        # Left paw velocity
        left_p_speeds = keypoint_speed(session, True, 'paw_r', False)
        left_p_times = left_p_speeds['left'][0][1:]
        left_p_speed = left_p_speeds['left'][1]

        # Left paw positions
        left_paw_X, left_paw_Y = left_paw(XYs)
        
        # Pupil velocity
        pupil_speeds = keypoint_speed(session, True, pupil_center, False)
        pupil_times = pupil_speeds['left'][0][1:]
        pupil_speed = pupil_speeds['left'][1]

        # Initialize dataframe
        licking = lick_rate(bin_size, session_licks, session_trials)
        wheel_vel = wheel_velocity(bin_size, wheel_times, pos, session_trials)
        wheel_disp = wheel_displacement(wheel_times, pos, session_trials, bin_size, onset_subtraction=False)
        pupil_dia = pupil_diam(dlc_times, pupil_dia_smooth, session_trials, bin_size, onset_subtraction=True)
        whisker_me = cont_bin(me_times, motion_energy, session_trials, bin_size)
        nose_vel = cont_bin(nose_times, nose_speed, session_trials, bin_size)
        left_vel = cont_bin(left_p_times, left_p_speed, session_trials, bin_size)
        left_X = cont_bin(dlc_times, left_paw_X, session_trials, bin_size)
        left_Y = cont_bin(dlc_times, left_paw_Y, session_trials, bin_size)
        pupil_vel = cont_bin(pupil_times, pupil_speed, session_trials, bin_size)
        pupil_X_pos = cont_bin(dlc_times, pupil_X, session_trials, bin_size)
        pupil_Y_pos = cont_bin(dlc_times, pupil_Y, session_trials, bin_size)
        
        # Rename column names
        pupil_dia = pupil_dia.rename(columns={'pupil_final':'pupil_diameter'})
        whisker_me = whisker_me.rename(columns={'Values':'whisker_me'})
        nose_vel = nose_vel.rename(columns={'Values':'nose_speed'})
        left_vel = left_vel.rename(columns={'Values':'l_paw_speed'})
        left_X = left_X.rename(columns={'Values':'left_X'})
        left_Y = left_Y.rename(columns={'Values':'left_Y'})
        pupil_vel = pupil_vel.rename(columns={'Values':'pupil_speed'})
        pupil_X_pos = pupil_X_pos.rename(columns={'Values':'pupil_X'})
        pupil_Y_pos = pupil_Y_pos.rename(columns={'Values':'pupil_Y'})     
        
        
        # Merge all tables
        all_metrics = wheel_vel[['Bin', 'avg_wheel_vel']].dropna().merge(licking[['Bin', 'Lick count']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(wheel_disp[['Bin', 'Onset times']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_dia[['Bin', 'pupil_diameter']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(whisker_me[['Bin', 'whisker_me']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_vel[['Bin', 'nose_speed']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_vel[['Bin', 'l_paw_speed']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_X[['Bin', 'left_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_Y[['Bin', 'left_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_vel[['Bin', 'pupil_speed']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_X_pos[['Bin', 'pupil_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_Y_pos[['Bin', 'pupil_Y']], on='Bin', how='outer')

        # Remove Nans before adding more data or else this breaks (not sure if this is really tru, but worked like this)
        all_metrics['Lick count'] = all_metrics['Lick count'].replace(np.NaN, 0)

    except:
        print(str(mouse_name + ' missing'))

/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:471: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:472: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/webclient.py:126: RuntimeWarning: Failed to connect, returning cached response
  warnings.warn('Failed to connect, returning cached response', RuntimeWarning)


Downloading: /Users/ineslaranjeira/Downloads/FlatIron/angelakilab/Subjects/NYU-40/2021-04-16/002/alf/leftCamera.ROIMotionEnergy.751b92f8-44b7-4081-86d2-3ce9cd31b9e0.npy Bytes: 1693288


100%|██████████| 1.6148452758789062/1.6148452758789062 [00:01<00:00,  1.48it/s]
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:471: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:472: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:471: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:472: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/autograd

NYU-46 missing
NYU-46 missing
NYU-47 missing
NYU-47 missing


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/webclient.py:126: RuntimeWarning: Failed to connect, returning cached response
  warnings.warn('Failed to connect, returning cached response', RuntimeWarning)


...

100%|██████████| 5.0543060302734375/5.0543060302734375 [00:01<00:00,  3.17it/s]
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/one/webclient.py:126: RuntimeWarning: Failed to connect, returning cached response
  warnings.warn('Failed to connect, returning cached response', RuntimeWarning)


Downloading: /Users/ineslaranjeira/Downloads/FlatIron/hausserlab/Subjects/PL017/2021-11-09/001/alf/_ibl_rightCamera.dlc.b39e2fea-5c35-4e8e-8324-f1071e49778f.pqt Bytes: 92166841


100%|██████████| 87.89714908599854/87.89714908599854 [00:12<00:00,  6.88it/s]
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:471: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:472: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:471: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:472: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/numpy/lib/

PL017 missing


/opt/anaconda3/envs/iblenv/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a, func=_nanmedian, keepdims=keepdims,
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:471: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/Users/ineslaranjeira/Documents/Repositories/representation_learning_variability/Functions/video_functions.py:472: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)


# Design matrix V2

In [4]:
for s, session in enumerate(pass_qc['session_uuid']):
    # TODO: as it is this takes ~10 minutes!! Mostly because of smoothing, outlier removal, ... of timeseries
    
    try:
        mouse_name = pass_qc['subject_nickname'][s]

        session_trials = one.load_object(session, obj='trials', namespace='ibl')
        session_trials = session_trials.to_df()

        # Licks
        session_licks = get_lick_times(session, combine=False, video_type=video_type)

        # Wheel
        wheel = one.load_object(session, 'wheel', collection='alf')
        pos, wheel_times = wh.interpolate_position(wheel.timestamps, wheel.position)

        # Pupil diameter
        pupil_dia_raw, pupil_dia_smooth, _, _ = (
            get_raw_and_smooth_position(session, video_type, ephys=True, position_function=get_pupil_diameter))
        dlc_times, XYs = get_dlc_XYs(session, view=video_type, likelihood_thresh=0.9)
        
        # Pupil position
        pupil_X, pupil_Y = pupil_center(XYs)
        
        # Motion energy of whisker pad
        me_times, motion_energy = get_ME(one, session, video_type)       
        #motion_energy = zscore(motion_energy, nan_policy='omit') 

        # Nose velocity
        nose_speeds = keypoint_speed(session, True, 'nose_tip', False)
        nose_times = nose_speeds['left'][0][1:]
        nose_speed = nose_speeds['left'][1]
        #nose_X, nose_Y = nose_tip(XYs)
        
        # Left paw velocity
        left_p_speeds = keypoint_speed(session, True, 'paw_r', False)
        left_p_times = left_p_speeds['left'][0][1:]
        left_p_speed = left_p_speeds['left'][1]

        # Left paw positions
        left_paw_X, left_paw_Y = left_paw(XYs)
        
        # Pupil velocity
        pupil_speeds = keypoint_speed(session, True, pupil_center, False)
        pupil_times = pupil_speeds['left'][0][1:]
        pupil_speed = pupil_speeds['left'][1]
        
        # Get session values
        session_length = list(session_trials['stimOff_times'][-1:])[0]  # TODO: this might not work if stimOff times are missing
        n_bins = int(np.floor(session_length/bin_size))
        onsets = session_trials['stimOn_times']

        # Initialize dataframe
        licking = lick_rate(bin_size, session_licks, session_trials)
        wheel_vel = wheel_velocity(bin_size, wheel_times, pos, session_trials)
        wheel_disp = wheel_displacement(wheel_times, pos, session_trials, bin_size, onset_subtraction=False)
        pupil_dia = pupil_diam(dlc_times, pupil_dia_smooth, session_trials, bin_size, onset_subtraction=True)
        whisker_me = cont_bin(me_times, motion_energy, session_trials, bin_size)
        nose_vel = cont_bin(nose_times, nose_speed, session_trials, bin_size)
        left_vel = cont_bin(left_p_times, left_p_speed, session_trials, bin_size)
        left_X = cont_bin(dlc_times, left_paw_X, session_trials, bin_size)
        left_Y = cont_bin(dlc_times, left_paw_Y, session_trials, bin_size)
        pupil_vel = cont_bin(pupil_times, pupil_speed, session_trials, bin_size)
        pupil_X_pos = cont_bin(dlc_times, pupil_X, session_trials, bin_size)
        pupil_Y_pos = cont_bin(dlc_times, pupil_Y, session_trials, bin_size)
        
        
        # Rename column names
        pupil_dia = pupil_dia.rename(columns={'pupil_final':'pupil_diameter'})
        whisker_me = whisker_me.rename(columns={'Values':'whisker_me'})
        nose_vel = nose_vel.rename(columns={'Values':'nose_speed'})
        left_vel = left_vel.rename(columns={'Values':'l_paw_speed'})
        left_X = left_X.rename(columns={'Values':'left_X'})
        left_Y = left_Y.rename(columns={'Values':'left_Y'})
        pupil_vel = pupil_vel.rename(columns={'Values':'pupil_speed'})
        pupil_X_pos = pupil_X_pos.rename(columns={'Values':'pupil_X'})
        pupil_Y_pos = pupil_Y_pos.rename(columns={'Values':'pupil_Y'})     
        
        
        # Merge all tables
        all_metrics = wheel_vel[['Bin', 'avg_wheel_vel']].dropna().merge(licking[['Bin', 'Lick count']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(wheel_disp[['Bin', 'Onset times']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_dia[['Bin', 'pupil_diameter']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(whisker_me[['Bin', 'whisker_me']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(nose_vel[['Bin', 'nose_speed']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_vel[['Bin', 'l_paw_speed']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_X[['Bin', 'left_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(left_Y[['Bin', 'left_Y']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_vel[['Bin', 'pupil_speed']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_X_pos[['Bin', 'pupil_X']], on='Bin', how='outer')
        all_metrics = all_metrics.merge(pupil_Y_pos[['Bin', 'pupil_Y']], on='Bin', how='outer')

        # Remove Nans before adding more data or else this breaks (not sure if this is really tru, but worked like this)
        all_metrics['Lick count'] = all_metrics['Lick count'].replace(np.NaN, 0)
        all_metrics = all_metrics.dropna().drop_duplicates()

        # Transform licks
        def anscombe_transform(x):
            return 2 * np.sqrt(x + 3/8)
        all_metrics['Gaussian_licks'] = anscombe_transform(all_metrics['Lick count'])
        all_metrics['Gaussian_licks'] = np.sqrt(all_metrics['Lick count'])

        # Save design matrix
        data_path =  '/home/ines/repositories/representation_learning_variability/DATA/Sub-trial/Design matrix/v2_3Nov2023/' + str(bin_size) + '/'
        os.chdir(data_path)
        pickle.dump(all_metrics, open("design_matrix_" + str(session) + '_'  + mouse_name + '_' + str(bin_size), "wb"))
        
    except:
       print(session)

/home/ines/miniconda3/envs/iblenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:455: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:456: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:455: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:456: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/home/ines/repositor

d32876dd-8303-4720-8e7e-20678dc2fd71


/home/ines/miniconda3/envs/iblenv/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:455: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:456: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:455: RuntimeWarning: Mean of empty slice
  X = np.nanmean(np.stack((l[:, 0], r[:, 0]), axis=0), axis=0)
/home/ines/repositories/representation_learning_variability/Functions/video_functions.py:456: RuntimeWarning: Mean of empty slice
  Y = np.nanmean(np.stack((t[:, 1], b[:, 1]), axis=0), axis=0)
/home/ines/repositor

72982282-e493-45ee-87ce-aa45cb3a3ec1


/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-24/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 3.34k/3.34k [00:00<00:00, 8.37kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-24/001/alf/_ibl_trials.intervals_bpod.npy: 100%|██████████| 6.54k/6.54k [00:00<00:00, 13.5kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-24/001/alf/_ibl_trials.table.pqt: 100%|██████████| 36.4k/36.4k [00:00<00:00, 78.1kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-24/001/alf/_ibl_trials.quiescencePeriod.npy: 100%|██████████| 3.34k/3.34k [00:00<00:00, 9.53kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelakilab/Subjects/NYU-46/2021-06-24/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 3.34k/3.34k [00:00<00:00, 7.73kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/angelaki

...

/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-19/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 3.94k/3.94k [00:00<00:00, 11.0kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-19/001/alf/_ibl_trials.quiescencePeriod.npy: 100%|██████████| 3.94k/3.94k [00:00<00:00, 11.8kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-19/001/alf/_ibl_trials.intervals_bpod.npy: 100%|██████████| 7.74k/7.74k [00:00<00:00, 22.6kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-19/001/alf/_ibl_trials.table.pqt: 100%|██████████| 41.3k/41.3k [00:00<00:00, 79.2kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-19/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 3.94k/3.94k [00:00<00:00, 11.0kB/s]
/home/ines/Downloads/ONE/alyx.internationalb

22e04698-b974-4805-b241-3b547dbf37bf


/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-22/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 5.39k/5.39k [00:00<00:00, 15.7kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-22/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 5.39k/5.39k [00:00<00:00, 14.8kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-22/001/alf/_ibl_trials.table.pqt: 100%|██████████| 53.7k/53.7k [00:00<00:00, 88.5kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-22/001/alf/_ibl_trials.quiescencePeriod.npy: 100%|██████████| 5.39k/5.39k [00:00<00:00, 15.4kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/mrsicflogellab/Subjects/SWC_052/2020-10-22/001/alf/_ibl_trials.intervals_bpod.npy: 100%|██████████| 10.7k/10.7k [00:00<00:00, 27.9kB/s]
/home/ines/Downloads/ONE/alyx.internationalb

09394481-8dd2-4d5c-9327-f2753ede92d7


/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/wittenlab/Subjects/ibl_witten_26/2021-01-31/001/alf/_ibl_trials.table.pqt: 100%|██████████| 84.5k/84.5k [00:00<00:00, 129kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/wittenlab/Subjects/ibl_witten_26/2021-01-31/001/alf/_ibl_trials.goCueTrigger_times.npy: 100%|██████████| 9.02k/9.02k [00:00<00:00, 24.4kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/wittenlab/Subjects/ibl_witten_26/2021-01-31/001/alf/_ibl_trials.stimOff_times.npy: 100%|██████████| 9.02k/9.02k [00:00<00:00, 25.5kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/wittenlab/Subjects/ibl_witten_26/2021-01-31/001/alf/_ibl_trials.quiescencePeriod.npy: 100%|██████████| 9.02k/9.02k [00:00<00:00, 26.6kB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/wittenlab/Subjects/ibl_witten_26/2021-01-31/001/alf/_ibl_trials.intervals_bpod.npy: 100%|██████████| 17.9k/17.9k [00:00<00:00, 53.4kB/s]
/home/ines/Downloads/ONE/alyx.internatio